In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

import utils

## Custom dense layer

In [2]:
class MyDense(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)

    def build(self, input_shape):
        self.kernel = self.add_weight(name='kernel', shape=[input_shape[-1], self.units], initializer='glorot_normal')
        self.bias = self.add_weight(name='bias', shape=[self.units], initializer='zeros')
        return super().build(input_shape)

    def call(self, inputs, **kwargs):
        return self.activation(inputs @ self.kernel + self.bias)

    def compute_output_shape(self, input_shape):
        return tf.TensorShape(input_shape.as_list()[-1] + [self.units])

    def get_config(self):
        base_config = super().get_config()
        return {
            **base_config,
            'units': self.units,
            'activation': keras.activations.serialize(self.activation),
        }

In [3]:
(x_train, y_train), (x_val, y_val), (x_test, y_test) = utils.load_housing_data()
print(f"x_train.shape = {x_train.shape}, y_train.shape = {y_train.shape}")
print(f"x_val.shape = {x_val.shape}, y_val.shape = {y_val.shape}")
print(f"x_test.shape = {x_test.shape}, y_test.shape = {y_test.shape}")

x_train.shape = (11610, 8), y_train.shape = (11610,)
x_val.shape = (3870, 8), y_val.shape = (3870,)
x_test.shape = (5160, 8), y_test.shape = (5160,)


In [4]:
utils.reset_session()

model = keras.models.Sequential([
    MyDense(30, activation='relu', input_shape=x_train.shape[1:]),
    MyDense(1)
])
model.summary()

model.compile(optimizer=keras.optimizers.SGD(lr=1e-3), loss='mse')

history = model.fit(x_train, y_train, epochs=10, validation_data=(x_val, y_val))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
my_dense (MyDense)           (None, 30)                270       
_________________________________________________________________
my_dense_1 (MyDense)         (None, 1)                 31        
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
363/363 [==============================] - 0s 835us/step - loss: 2.3168 - val_loss: 1.2389
Epoch 2/10
363/363 [==============================] - 0s 617us/step - loss: 0.7808 - val_loss: 0.7033
Epoch 3/10
363/363 [==============================] - 0s 620us/step - loss: 0.6809 - val_loss: 0.6434
Epoch 4/10
363/363 [==============================] - 0s 617us/step - loss: 0.6314 - val_loss: 0.6128
Epoch 5/10
363/363 [==============================] - 0s 623us/step - loss: 0.5932 - val_loss: 0.5734
Epoch 6/10

In [5]:
model.evaluate(x_val, y_val)

121/121 [==============================] - 0s 422us/step - loss: 0.4675


0.46753329038619995

In [6]:
model.save('custom_dense.h5')

In [7]:
utils.reset_session()
keras.models.load_model('custom_dense.h5', custom_objects={"MyDense": MyDense})
model.evaluate(x_val, y_val)

121/121 [==============================] - 0s 446us/step - loss: 0.4675


0.46753329038619995

## Different behavior during training and testing

In [8]:
class AddGaussianNoise(keras.layers.Layer):
    def __init__(self, stddev, **kwargs):
        super().__init__(**kwargs)
        self.stddev = stddev

    def call(self, inputs, **kwargs):
        training = kwargs.get('training')
        print(f"AddGaussianNoise call(): training = {training}, kwargs = {kwargs}")
        if training:
            noise = tf.random.normal(tf.shape(inputs), stddev=self.stddev)
            return inputs + noise
        else:
            return inputs

    def compute_output_shape(self, input_shape):
        return input_shape

In [9]:
utils.reset_session()

model = keras.models.Sequential([
    MyDense(30, activation='relu', input_shape=x_train.shape[1:]),
    AddGaussianNoise(0.1),
    MyDense(1)
])
model.summary()

model.compile(optimizer=keras.optimizers.SGD(lr=1e-3), loss='mse')

history = model.fit(x_train, y_train, epochs=10, validation_data=(x_train, y_train))

AddGaussianNoise call(): training = None, kwargs = {'training': None}
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
my_dense (MyDense)           (None, 30)                270       
_________________________________________________________________
add_gaussian_noise (AddGauss (None, 30)                0         
_________________________________________________________________
my_dense_1 (MyDense)         (None, 1)                 31        
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
AddGaussianNoise call(): training = True, kwargs = {'training': True}
AddGaussianNoise call(): training = True, kwargs = {'training': True}
363/363 [==============================] - 0s 1ms/step - loss: 2.3303 - val_loss: 0.8962
Epoch 2/10
363/363 [==============================] - 0s 865us/step - loss: 0.

In [10]:
model.evaluate(x_train, y_train)

363/363 [==============================] - 0s 398us/step - loss: 0.4811


0.48112058639526367

## Custom layer with multiple inputs

In [11]:
class MultipleInputLayer(keras.layers.Layer):
    def call(self, inputs, **kwargs):
        x1, x2 = inputs
        return x1 + x2, x1 * x2, x1 / x2

    def compute_output_shape(self, input_shape):
        s1, s2 = input_shape
        return [s1, s1, s1]

In [12]:
layer = MultipleInputLayer()

In [13]:
a = tf.constant([1, 2])
b = tf.constant([3, 4])

In [14]:
layer([a, b])

(<tf.Tensor: shape=(2,), dtype=int32, numpy=array([4, 6])>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([3, 8])>,
 <tf.Tensor: shape=(2,), dtype=float64, numpy=array([0.33333333, 0.5       ])>)